# Combines results across chromosomes

Post-run program to merge tables (summary and expanded table) across chromosomes to enable genome-wide analysis

Instructions:
1. Run each cell from the beginning in order
2. Modify the variables where suggested

In [47]:
import pandas as pd
import os
import glob
import re

In [48]:
def _chrom_order(chrom):
    """
    Map chr names to an integer order
    """
    m = re.match(r'^chr(\d+)$', chrom)
    if m:
        return int(m.group(1))
    if chrom in ('chrX', 'X'):
        return 23
    if chrom in ('chrY', 'Y'):
        return 24
    if chrom in ('chrM', 'chrMT', 'MT'):
        return 25
    # put everything else at the end
    return float('inf')

In [49]:
def extract_comments(csv_path: str) -> list[str]:
    """
    Read the leading comments “#” from a csv and return them 
    Stops as soon as it hits a non-# line
    """
    comments = []
    with open(csv_path, 'r') as fh:
        for line in fh:
            if line.startswith('#'):
                # strip newline, then replace chr<number> with chr*
                txt = line.rstrip('\n')
                txt = re.sub(r'chr\d+', 'chr*', txt)
                comments.append(txt)
            else:
                break
    return comments

# Repli-HiC_K562_WT_totalS_chr_combinedKb

def combine_results(results_dir, folder_pattern, result_type, exp_name=None):
    """
    Search for folders under `results_dir` matching `folder_pattern` (which may
    contain a glob-style asterisk), then for each folder:
      - load <folder>/<folder>_results_<result_type>/<folder>_summary_table.csv
      - load <folder>/<folder>_results_<result_type>/<folder>_expanded_table.csv
    and concatenate all summaries and all expanded tables into two DataFrames.

    Parameters
    ----------
    results_dir : str
        Path to the directory containing all result‐folders
    folder_pattern : str
        A glob style pattern (e.g. "splenic-B-cell_*_50Kb") used to match subfolders
    result_type : str
        The suffix after "_results" in each folder (e.g. "all", "p-0.01", etc)

    Returns
    -------
    (combined_summary, combined_expanded) : tuple of pd.DataFrame
        The concatenated summary and expanded tables
    """
    # build the full glob pattern
    search_pattern = os.path.join(results_dir, folder_pattern)

    prefix, suffix = folder_pattern.split('*')

    if exp_name is not None:
        exp_prefix, exp_suffix = exp_name.split('*')
        folder_regex = re.compile(
            rf"^{re.escape(prefix)}(.+?){re.escape(suffix)}$"
        )

    # If this notebook is already run, then do not include the (already combined) combined folder
    exclude_name = f"{prefix}_combined{suffix}"
    print(exclude_name)

    pattern = re.compile(
        rf"^{re.escape(prefix)}"      # “Repli-HiC_K562_WT_totalS_chr”
        r".+" # Changed to match any character because ce10 genome is roman numerals
        rf"{re.escape(suffix)}$"      # “_50Kb”
    )

    matched_folders = [
        os.path.join(results_dir, d)
        for d in os.listdir(results_dir)
        if pattern.match(d)
    ]

    # Exclude the combined folder if it exists
    matched_folders = [
        f for f in matched_folders
        if os.path.basename(f) != exclude_name
    ]

    if not matched_folders:
        raise FileNotFoundError(f"No folders found matching {search_pattern}")

    summary_frames = []
    expanded_frames = []
    bedpe_frames = []
    for i, folder in enumerate(matched_folders):

        if exp_name is not None:
            m = folder_regex.match(os.path.basename(folder))
            if not m:
                raise ValueError(
                    f"Folder name '{os.path.basename(folder)}' "
                    f"did not match pattern '{folder_pattern}'"
                )
            wild = m.group(1)
            base = f"{exp_prefix}{wild}{exp_suffix}"
        else:
            # basename is the actual folder name without the path
            base = os.path.basename(folder)

        # path to the results‐type subdirectory
        res_subdir = os.path.join(folder, f"{base}_results_{result_type}")

        # csv paths
        summary_csv = os.path.join(res_subdir, f"{base}_summary_table.csv")
        expanded_csv = os.path.join(res_subdir, f"{base}_expanded_table.csv")
        juicer_bedpe = os.path.join(res_subdir, f"{base}_juicer-visualize.bedpe")

        # Load and collect
        if not os.path.isfile(summary_csv):
            raise FileNotFoundError(f"Expected file not found: {summary_csv}")
        if not os.path.isfile(expanded_csv):
            raise FileNotFoundError(f"Expected file not found: {expanded_csv}")
        if not os.path.isfile(juicer_bedpe):
            raise FileNotFoundError(f"Expected file not found: {juicer_bedpe}")
        
        # extract the comments from a single summary csv only
        if i == 0:
            comments = extract_comments(summary_csv)
            for comment in comments:
                print(comment)

        summary_frames.append(pd.read_csv(summary_csv, comment='#'))
        expanded_frames.append(pd.read_csv(expanded_csv, comment='#'))
        bedpe_frames.append(pd.read_csv(juicer_bedpe, comment='#', sep='\t', header=None, index_col=False, 
                                        names=['chrom1', 'start1', 'end1', 'chrom2', 'start2', 'end2']))

    # Concatenate all and reset the index
    combined_summary = pd.concat(summary_frames,  ignore_index=True)
    combined_expanded = pd.concat(expanded_frames, ignore_index=True)
    combined_bedpe = pd.concat(bedpe_frames, ignore_index=True)

    # Sort 
    # First convert chromosomes to a numerical value for sorting
    combined_summary["chrom_order"] = combined_summary["chrom"].apply(_chrom_order)
    combined_expanded["chrom_order"] = combined_expanded["chrom"].apply(_chrom_order)
    combined_bedpe["chrom1_order"] = combined_bedpe["chrom1"].apply(_chrom_order)
    combined_bedpe["chrom2_order"] = combined_bedpe["chrom2"].apply(_chrom_order)
    # Then simply sort by chromosome order and position
    combined_summary.sort_values(by=["chrom_order"], inplace=True)
    combined_expanded.sort_values(by=["chrom_order"], inplace=True)
    combined_bedpe.sort_values(by=["chrom1_order", "start1", "chrom2_order", "start2"], inplace=True)
    # Drop the temporary chrom_order column 
    combined_summary.drop(columns=["chrom_order"], inplace=True)
    combined_expanded.drop(columns=["chrom_order"], inplace=True)
    combined_bedpe.drop(columns=["chrom1_order", "chrom2_order"], inplace=True)

    # Reset index 
    combined_summary.reset_index(drop=True, inplace=True)
    combined_expanded.reset_index(drop=True, inplace=True)
    combined_bedpe.reset_index(drop=True, inplace=True)

    return combined_summary, combined_expanded, combined_bedpe, comments

# Save the combined tables
def save_csv(df, save_dir, parameter_str_comment):
    """
    Save a dataframe to a csv file 

    Parameters
    ----------
    df : pd.DataFrame
        The input dataframe to save
    save_dir : str
        The name of the file to save the dataframe to
    parameter_str : list
        A list containing parameters to be included in the beginning of the csv file as comments
        Each element in the list is a string that must start with a special character (e.g '#')
    
    Returns
    -------
    None, but saves the dataframe to a csv file
    """
    with open(save_dir, "w") as f:
        f.write("\n".join(parameter_str_comment) + "\n")
        df.to_csv(f, index=False)

# Individual

**MODIFY the following cell as needed**

In [50]:
# Let the notebook know what experiment type you would like to combine
# Specify it as a folder pattern with an asterisk * to match multiple directories
# Example: "splenic-B-cell_WT_insitu-hic_GSE82144_mm9_chr*_50Kb" will match all chromosomes
# of that specific experiment run
# folder_pattern = "splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr*_50Kb"
# folder_pattern = "DP-thymocytes_WT_hic_Guo-2022_GSE199059_mm10-remapped_chr*_50Kb"
# folder_pattern = "GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb"
# folder_pattern = "Repli-HiC_K562_WT_totalS_chr*_50Kb"
folder_pattern = "GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb"
# folder_pattern = "GSE199059_CD69negDPWTR1R2R3R4_merged-normalized-2_chr*_50Kb"
# folder_pattern = "GSE199059_CD69negDPWTR1R2R3R4_merged-root-2_chr*_50Kb"
# folder_pattern = "Repli-HiC_K562_WT_totalS-stripiness-1_chr*_50Kb"

# exp_name = None
exp_name = "GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb" 
# exp_name = "Repli-HiC_K562_WT_totalS_chr*_50Kb"

# Location of the results folders
# Where is the folder above located?
results_dir = "/nfs/turbo/umms-minjilab/sionkim/miajet_output/"

# This is the suffix after the "_results" in each folder 
# You can specify "all" to combine all results or a specifically thresholded output
# Examples: "all", "p-0.01", "saliency-90", "saliency-90-p-0.1"
result_type = ["saliency-90-p-0.1", "saliency-90-p-0.05", "saliency-90-p-0.01", "saliency-90", "p-0.1", "p-0.05", "p-0.01", "all"]

In [51]:
if isinstance(result_type, str):
    df_s, df_e, df_b, parameter_str = combine_results(results_dir, folder_pattern, result_type)
    # Will print out the parametesr of the experiment
    # So that you can confirm this is indeed the correct experiment you are combining

    df_summary = [df_s]
    df_expanded = [df_e]
    df_bedpe = [df_b]
else:
    df_summary = []
    df_expanded = []
    df_bedpe = []
    for rt in result_type:
        df_s, df_e, df_b, parameter_str = combine_results(results_dir, folder_pattern, rt, exp_name=exp_name)
        df_summary.append(df_s)
        df_expanded.append(df_e)
        df_bedpe.append(df_b)

GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_50Kb
# GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/GSE199059_CD69negDPWTR1R2R3R4_merged.hic
# 
# Required Parameters
# * Experiment type: hic
# * Chromosome: chr*
# * Resolution: 50000
# * Save directory root: /nfs/turbo/umms-minjilab/sionkim/miajet_output
# 
# Extended Parameters
# * Significance threshold(s): [0.1, 0.05, 0.01]
# * Window size: 6000000
# * Normalization: KR
# * Data type: oe
# * Jet widths (if specified): None
# * Angle range: [80, 100]
# * Saliency threshold on zero removed: 90
# * Hysteresis thresholding parameters: [0.01, 0.05]
# * Root within: 10
# * Folder name: GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb
# * Save directory: /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb
# * Number of cores: 4
# * Verbose: True
# 


/tmp/ipykernel_844100/1278568343.py:124: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_summary = pd.concat(summary_frames,  ignore_index=True)
/tmp/ipykernel_844100/1278568343.py:125: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_expanded = pd.concat(expanded_frames, ignore_index=True)


GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_50Kb
# GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/GSE199059_CD69negDPWTR1R2R3R4_merged.hic
# 
# Required Parameters
# * Experiment type: hic
# * Chromosome: chr*
# * Resolution: 50000
# * Save directory root: /nfs/turbo/umms-minjilab/sionkim/miajet_output
# 
# Extended Parameters
# * Significance threshold(s): [0.1, 0.05, 0.01]
# * Window size: 6000000
# * Normalization: KR
# * Data type: oe
# * Jet widths (if specified): None
# * Angle range: [80, 100]
# * Saliency threshold on zero removed: 90
# * Hysteresis thresholding parameters: [0.01, 0.05]
# * Root within: 10
# * Folder name: GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb
# * Save directory: /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb
# * Number of cores: 4
# * Verbose: True
# 


/tmp/ipykernel_844100/1278568343.py:124: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_summary = pd.concat(summary_frames,  ignore_index=True)
/tmp/ipykernel_844100/1278568343.py:125: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_expanded = pd.concat(expanded_frames, ignore_index=True)


GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_50Kb
# GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/GSE199059_CD69negDPWTR1R2R3R4_merged.hic
# 
# Required Parameters
# * Experiment type: hic
# * Chromosome: chr*
# * Resolution: 50000
# * Save directory root: /nfs/turbo/umms-minjilab/sionkim/miajet_output
# 
# Extended Parameters
# * Significance threshold(s): [0.1, 0.05, 0.01]
# * Window size: 6000000
# * Normalization: KR
# * Data type: oe
# * Jet widths (if specified): None
# * Angle range: [80, 100]
# * Saliency threshold on zero removed: 90
# * Hysteresis thresholding parameters: [0.01, 0.05]
# * Root within: 10
# * Folder name: GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb
# * Save directory: /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb
# * Number of cores: 4
# * Verbose: True
# 


...

GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_50Kb
# GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/GSE199059_CD69negDPWTR1R2R3R4_merged.hic
# 
# Required Parameters
# * Experiment type: hic
# * Chromosome: chr*
# * Resolution: 50000
# * Save directory root: /nfs/turbo/umms-minjilab/sionkim/miajet_output
# 
# Extended Parameters
# * Significance threshold(s): [0.1, 0.05, 0.01]
# * Window size: 6000000
# * Normalization: KR
# * Data type: oe
# * Jet widths (if specified): None
# * Angle range: [80, 100]
# * Saliency threshold on zero removed: 90
# * Hysteresis thresholding parameters: [0.01, 0.05]
# * Root within: 10
# * Folder name: GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb
# * Save directory: /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb
# * Number of cores: 4
# * Verbose: True
# 


/tmp/ipykernel_844100/1278568343.py:124: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_summary = pd.concat(summary_frames,  ignore_index=True)
/tmp/ipykernel_844100/1278568343.py:125: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_expanded = pd.concat(expanded_frames, ignore_index=True)


GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_50Kb
# GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/GSE199059_CD69negDPWTR1R2R3R4_merged.hic
# 
# Required Parameters
# * Experiment type: hic
# * Chromosome: chr*
# * Resolution: 50000
# * Save directory root: /nfs/turbo/umms-minjilab/sionkim/miajet_output
# 
# Extended Parameters
# * Significance threshold(s): [0.1, 0.05, 0.01]
# * Window size: 6000000
# * Normalization: KR
# * Data type: oe
# * Jet widths (if specified): None
# * Angle range: [80, 100]
# * Saliency threshold on zero removed: 90
# * Hysteresis thresholding parameters: [0.01, 0.05]
# * Root within: 10
# * Folder name: GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb
# * Save directory: /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb
# * Number of cores: 4
# * Verbose: True
# 


/tmp/ipykernel_844100/1278568343.py:124: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_summary = pd.concat(summary_frames,  ignore_index=True)
/tmp/ipykernel_844100/1278568343.py:125: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_expanded = pd.concat(expanded_frames, ignore_index=True)


GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_50Kb
# GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/GSE199059_CD69negDPWTR1R2R3R4_merged.hic
# 
# Required Parameters
# * Experiment type: hic
# * Chromosome: chr*
# * Resolution: 50000
# * Save directory root: /nfs/turbo/umms-minjilab/sionkim/miajet_output
# 
# Extended Parameters
# * Significance threshold(s): [0.1, 0.05, 0.01]
# * Window size: 6000000
# * Normalization: KR
# * Data type: oe
# * Jet widths (if specified): None
# * Angle range: [80, 100]
# * Saliency threshold on zero removed: 90
# * Hysteresis thresholding parameters: [0.01, 0.05]
# * Root within: 10
# * Folder name: GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb
# * Save directory: /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb
# * Number of cores: 4
# * Verbose: True
# 


In [52]:
# Can visually inspect the summary table
df_summary[0]

,unique_id,chrom,start,end,length,input_mean,angle_mean,width_mean,jet_saliency,ks,p-val_raw,p-val_corr,stripiness
0,chr2_3_11,chr2,9.517042e+06,1.020590e+07,1150000,0.254,99.127,7.354,0.072,0.826,0.000,0.000,0.070
1,chr2_94_6,chr2,1.662365e+08,1.663939e+08,300000,0.230,89.441,4.017,0.075,1.000,0.001,0.005,0.027
2,chr2_1254_12,chr2,4.481502e+07,4.533418e+07,800000,0.245,92.369,6.748,0.298,0.938,0.000,0.000,0.073
3,chr2_751_17,chr2,1.138397e+07,1.218821e+07,1250000,0.192,84.599,9.265,0.110,0.600,0.000,0.001,0.026
4,chr2_131_10,chr2,1.155614e+08,1.161160e+08,1150000,0.165,113.008,4.801,0.099,0.478,0.005,0.017,0.013
5,chr2_7701_0,chr2,7.905646e+07,7.951752e+07,650000,0.242,67.506,2.507,0.078,0.692,0.001,0.007,0.025
6,chr3_2404_5,chr3,1.195792e+08,1.201041e+08,850000,0.192,91.922,4.956,0.242,0.765,0.000,0.000,0.029
7,chr3_10_14,chr3,5.661763e+06,6.623542e+06,1500000,0.275,93.320,7.417,0.516,0.933,0.000,0.000,0.057
8,chr3_7_14,chr3,3.790186e+07,3.868163e+07,1100000,0.298,66.409,7.112,0.138,0.500,0.004,0.024,0.084
9,chr5_109_0,chr5,1.502087e+08,1.503996e+08,600000,0.057,112.699,3.522,0.014,0.917,0.000,0.000,0.002


In [53]:
# Can visually inspect the expanded table
df_expanded[0]

,unique_id,chrom,x (bp),y (bp),x (pixels),y (pixels),width,angle_imagej,ridge_strength
0,chr2_1254_12,chr2,4.484858e+07,4.467702e+07,1266.497,81.719,5.776,90.830,0.017
1,chr2_1254_12,chr2,4.488184e+07,4.464002e+07,1266.444,80.726,5.680,91.270,0.019
2,chr2_3_11,chr2,9.726641e+06,9.107299e+06,266.766,75.387,6.960,113.970,0.026
3,chr2_3_11,chr2,9.738771e+06,9.064294e+06,266.330,74.607,7.284,112.118,0.025
4,chr2_3_11,chr2,9.742604e+06,8.999958e+06,265.474,73.643,8.332,105.103,0.024
...,...,...,...,...,...,...,...,...,...
593,chrX_326_8,chrX,4.985425e+07,4.863327e+07,1393.238,66.878,4.798,93.318,0.002
594,chrX_326_8,chrX,4.995339e+07,4.852310e+07,1393.082,63.918,4.875,93.648,0.002
595,chrX_69_8,chrX,1.393669e+08,1.387090e+08,3933.002,74.841,5.808,95.109,0.003
596,chrX_69_8,chrX,1.394380e+08,1.386374e+08,3932.995,72.824,4.404,90.560,0.003


In [54]:
# Can visually inspect the bedpe table
df_bedpe[0]

,chrom1,start1,end1,chrom2,start2,end2
0,chr2,8635403,8660403,chr2,10205898,10230898
1,chr2,8655745,8680745,chr2,10163394,10188394
2,chr2,8680510,8705510,chr2,10106384,10131384
3,chr2,8702482,8727482,chr2,10064133,10089133
4,chr2,8724640,8749640,chr2,10021337,10046337
...,...,...,...,...,...,...
593,chrX,138602009,138627009,chrX,139473184,139498184
594,chrX,138637444,138662444,chrX,139438026,139463026
595,chrX,138673077,138698077,chrX,139402738,139427738
596,chrX,138708986,138733986,chrX,139366932,139391932


**(OPTIONAL) modify as needed**

In [55]:
save_name = folder_pattern.replace('*', f'_combined') 
# This is the save name for the combined results

save_dir = os.path.join(results_dir, save_name)
# Folder to save the combined results (summary and expanded tables)
# You may overwrite the `results_dir` if you wish to save to a different location

print("Saving combined tables to:")
print(f"* Folder location: {save_dir}")

save_dir_summary = []
save_dir_expanded = []
save_dir_bedpe = []
for rt in result_type:
    print(f"* Result type: {rt}")
    save_name_file = folder_pattern.replace('*', f'_combined_{rt}')

    # Below is the code to print out the save locations to help you modify the variables abovef
    # Once satisfied, move on to the next cell
    save_dir_s = os.path.join(save_dir, f"{save_name_file}_summary_table.csv")
    save_dir_e = os.path.join(save_dir, f"{save_name_file}_expanded_table.csv")
    save_dir_b = os.path.join(save_dir, f"{save_name_file}_juicer-visualize.bedpe")

    save_dir_summary.append(save_dir_s)
    save_dir_expanded.append(save_dir_e)
    save_dir_bedpe.append(save_dir_b)

    print(f"\t* Summary table: {save_name_file}_summary_table.csv")
    print(f"\t* Expanded table: {save_name_file}_expanded_table.csv")
    print(f"\t* Bedpe table: {save_name_file}_juicer-visualize.bedpe")

Saving combined tables to:
* Folder location: /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_50Kb
* Result type: saliency-90-p-0.1
	* Summary table: GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_saliency-90-p-0.1_50Kb_summary_table.csv
	* Expanded table: GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_saliency-90-p-0.1_50Kb_expanded_table.csv
	* Bedpe table: GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_saliency-90-p-0.1_50Kb_juicer-visualize.bedpe
* Result type: saliency-90-p-0.05
	* Summary table: GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_saliency-90-p-0.05_50Kb_summary_table.csv
	* Expanded table: GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_saliency-90-p-0.05_50Kb_expanded_table.csv
	* Bedpe table: GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-

In [56]:
# Make directory if it doesn't exist already and save the files
os.makedirs(save_dir, exist_ok=True)

for i in range(len(result_type)):
    save_csv(df_summary[i], save_dir_summary[i], parameter_str)
    save_csv(df_expanded[i], save_dir_expanded[i], parameter_str)
    df_bedpe[i].to_csv(save_dir_bedpe[i], sep='\t', index=False, header=False) 
    print(f"Saved\n *{save_dir_summary[i]}\n *{save_dir_expanded[i]}\n *{save_dir_bedpe[i]}")

Saved
 */nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_saliency-90-p-0.1_50Kb_summary_table.csv
 */nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_saliency-90-p-0.1_50Kb_expanded_table.csv
 */nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_saliency-90-p-0.1_50Kb_juicer-visualize.bedpe
Saved
 */nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr_combined_saliency-90-p-0.05_50Kb_summary_tab

# Multiple samples


Loads in the `submit_all_config.yaml` used to submit batch jobs on Slurm to now combine the results

In [19]:
import yaml

import sys
sys.path.append('..')
from utils.plotting import genomic_labels

In [20]:
with open("../submit_all_config.yaml") as cf:
    config = yaml.safe_load(cf)

samples = config["samples"]

results_dir = "/nfs/turbo/umms-minjilab/sionkim/miajet_output/"
result_types = [
    "saliency-90-p-0.1", "saliency-90-p-0.05", "saliency-90-p-0.01",
    "saliency-90", "p-0.1", "p-0.05", "p-0.01", "all"
]

for key, attributes in samples.items():
    print(f"Processing experiment: {key}")

    f = attributes["file"]
    basename = os.path.basename(f)
    exp = os.path.splitext(basename)[0]  # Remove file extension

    resolution = genomic_labels(int(attributes["res"]))

    folder_pattern = f"{exp}_chr*_{resolution}"
    save_name = folder_pattern.replace('*', '_combined')
    save_dir = os.path.join(results_dir, save_name)
    os.makedirs(save_dir, exist_ok=True)

    # Combine results for each result type
    df_summary_list = []
    df_expanded_list = []
    df_bedpe_list   = []
    for rt in result_types:
        print(f"\tCombining result type: {rt}")
        df_s, df_e, df_b, parameter_str = combine_results(results_dir, folder_pattern, rt)
        df_summary_list.append(df_s)
        df_expanded_list.append(df_e)
        df_bedpe_list.append(df_b)

        # Construct file paths
        save_base = folder_pattern.replace('*', f"_combined_{rt}")
        summary_fp = os.path.join(save_dir, f"{save_base}_summary_table.csv")
        expanded_fp = os.path.join(save_dir, f"{save_base}_expanded_table.csv")
        bedpe_fp = os.path.join(save_dir, f"{save_base}_juicer-visualize.bedpe")

        # Save outputs
        save_csv(df_s, summary_fp, parameter_str)
        save_csv(df_e, expanded_fp, parameter_str)
        df_b.to_csv(bedpe_fp, sep='\t', index=False, header=False)

        print(f"\tSaved summary:  {summary_fp}")
        print(f"\tSaved expanded: {expanded_fp}")
        print(f"\tSaved BEDPE:    {bedpe_fp}")

    print()  


Processing experiment: Repli-HiC_K562_WT_totalS
	Combining result type: saliency-90-p-0.1
Repli-HiC_K562_WT_totalS_chr_combined_50Kb
# Repli-HiC_K562_WT_totalS_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/Repli-HiC_K562_WT_totalS.hic
# 
# Required Parameters
# * Experiment type: replihic
# * Chromosome: chr*
# * Resolution: 50000
# * Save directory root: /nfs/turbo/umms-minjilab/sionkim/miajet_output
# 
# Extended Parameters
# * Significance threshold(s): [0.1, 0.05, 0.01]
# * Window size: 6000000
# * Normalization: VC_SQRT
# * Data type: observed
# * Jet widths (if specified): None
# * Angle range: [80, 100]
# * Saliency threshold on zero removed: 90
# * Hysteresis thresholding parameters: [0.01, 0.05]
# * Root within: None
# * Folder name: Repli-HiC_K562_WT_totalS_chr*_50Kb
# * Save directory: /nfs/turbo/umms-minjilab/sionkim/miajet_output/Repli-HiC_K562_WT_totalS_chr*_50Kb
# * Number of cores: 4
# * Verbose: True
# 
# Optional Parameters
# * Scal

/tmp/ipykernel_3791577/3663817091.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_summary = pd.concat(summary_frames,  ignore_index=True)
/tmp/ipykernel_3791577/3663817091.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_expanded = pd.concat(expanded_frames, ignore_index=True)


	Saved summary:  /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_saliency-90-p-0.01_50Kb_summary_table.csv
	Saved expanded: /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_saliency-90-p-0.01_50Kb_expanded_table.csv
	Saved BEDPE:    /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_saliency-90-p-0.01_50Kb_juicer-visualize.bedpe
	Combining result type: saliency-90
GSE199059_CD69negDPWTR1R2R3R4_merged_chr_combined_50Kb
# GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/GSE199059_CD69negDPWTR1R2R3R4_merged.hic
# 
# Required Parameters
# * Experiment type: hic
# * Chromosome: chr*
# * Resolution: 50000
# * Save direct

/tmp/ipykernel_3791577/3663817091.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_summary = pd.concat(summary_frames,  ignore_index=True)
/tmp/ipykernel_3791577/3663817091.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_expanded = pd.concat(expanded_frames, ignore_index=True)


	Saved summary:  /nfs/turbo/umms-minjilab/sionkim/miajet_output/GM12878_insitu-hic_4DNFI1UEG1HD_chr_combined_50Kb/GM12878_insitu-hic_4DNFI1UEG1HD_chr_combined_saliency-90-p-0.01_50Kb_summary_table.csv
	Saved expanded: /nfs/turbo/umms-minjilab/sionkim/miajet_output/GM12878_insitu-hic_4DNFI1UEG1HD_chr_combined_50Kb/GM12878_insitu-hic_4DNFI1UEG1HD_chr_combined_saliency-90-p-0.01_50Kb_expanded_table.csv
	Saved BEDPE:    /nfs/turbo/umms-minjilab/sionkim/miajet_output/GM12878_insitu-hic_4DNFI1UEG1HD_chr_combined_50Kb/GM12878_insitu-hic_4DNFI1UEG1HD_chr_combined_saliency-90-p-0.01_50Kb_juicer-visualize.bedpe
	Combining result type: saliency-90
GM12878_insitu-hic_4DNFI1UEG1HD_chr_combined_50Kb
# GM12878_insitu-hic_4DNFI1UEG1HD_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/GM12878_insitu-hic_4DNFI1UEG1HD.hic
# 
# Required Parameters
# * Experiment type: hic
# * Chromosome: chr*
# * Resolution: 50000
# * Save directory root: /nfs/turbo/umms-minjilab/sionkim/mi

...

	Saved summary:  /nfs/turbo/umms-minjilab/sionkim/miajet_output/c-elegans-JK06-L3_WAPL-auxin-1hr_hic_Kim-2023_GSE237663_ce10_chr_combined_40Kb/c-elegans-JK06-L3_WAPL-auxin-1hr_hic_Kim-2023_GSE237663_ce10_chr_combined_saliency-90_40Kb_summary_table.csv
	Saved expanded: /nfs/turbo/umms-minjilab/sionkim/miajet_output/c-elegans-JK06-L3_WAPL-auxin-1hr_hic_Kim-2023_GSE237663_ce10_chr_combined_40Kb/c-elegans-JK06-L3_WAPL-auxin-1hr_hic_Kim-2023_GSE237663_ce10_chr_combined_saliency-90_40Kb_expanded_table.csv
	Saved BEDPE:    /nfs/turbo/umms-minjilab/sionkim/miajet_output/c-elegans-JK06-L3_WAPL-auxin-1hr_hic_Kim-2023_GSE237663_ce10_chr_combined_40Kb/c-elegans-JK06-L3_WAPL-auxin-1hr_hic_Kim-2023_GSE237663_ce10_chr_combined_saliency-90_40Kb_juicer-visualize.bedpe
	Combining result type: p-0.1
c-elegans-JK06-L3_WAPL-auxin-1hr_hic_Kim-2023_GSE237663_ce10_chr_combined_40Kb
# c-elegans-JK06-L3_WAPL-auxin-1hr_hic_Kim-2023_GSE237663_ce10_chrV_40Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/

/tmp/ipykernel_3791577/3663817091.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_summary = pd.concat(summary_frames,  ignore_index=True)
/tmp/ipykernel_3791577/3663817091.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_expanded = pd.concat(expanded_frames, ignore_index=True)


	Saved summary:  /nfs/turbo/umms-minjilab/sionkim/miajet_output/GM12878_intacthic_ENCFF318GOM_chr_combined_50Kb/GM12878_intacthic_ENCFF318GOM_chr_combined_saliency-90-p-0.01_50Kb_summary_table.csv
	Saved expanded: /nfs/turbo/umms-minjilab/sionkim/miajet_output/GM12878_intacthic_ENCFF318GOM_chr_combined_50Kb/GM12878_intacthic_ENCFF318GOM_chr_combined_saliency-90-p-0.01_50Kb_expanded_table.csv
	Saved BEDPE:    /nfs/turbo/umms-minjilab/sionkim/miajet_output/GM12878_intacthic_ENCFF318GOM_chr_combined_50Kb/GM12878_intacthic_ENCFF318GOM_chr_combined_saliency-90-p-0.01_50Kb_juicer-visualize.bedpe
	Combining result type: saliency-90
GM12878_intacthic_ENCFF318GOM_chr_combined_50Kb
# GM12878_intacthic_ENCFF318GOM_chr*_50Kb
# Inputs
# * Hi-C file (.hic): /nfs/turbo/umms-minjilab/downloaded_data/GM12878_intacthic_ENCFF318GOM.hic
# 
# Required Parameters
# * Experiment type: hic
# * Chromosome: chr*
# * Resolution: 50000
# * Save directory root: /nfs/turbo/umms-minjilab/sionkim/miajet_output
# 
# E

/tmp/ipykernel_3791577/3663817091.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_summary = pd.concat(summary_frames,  ignore_index=True)
/tmp/ipykernel_3791577/3663817091.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_expanded = pd.concat(expanded_frames, ignore_index=True)


	Saved summary:  /nfs/turbo/umms-minjilab/sionkim/miajet_output/HCT116_RAD21-auxin-0hr_intacthic_Guckelberger-2024_ENCFF528XGK_hg38_chr_combined_50Kb/HCT116_RAD21-auxin-0hr_intacthic_Guckelberger-2024_ENCFF528XGK_hg38_chr_combined_saliency-90-p-0.01_50Kb_summary_table.csv
	Saved expanded: /nfs/turbo/umms-minjilab/sionkim/miajet_output/HCT116_RAD21-auxin-0hr_intacthic_Guckelberger-2024_ENCFF528XGK_hg38_chr_combined_50Kb/HCT116_RAD21-auxin-0hr_intacthic_Guckelberger-2024_ENCFF528XGK_hg38_chr_combined_saliency-90-p-0.01_50Kb_expanded_table.csv
	Saved BEDPE:    /nfs/turbo/umms-minjilab/sionkim/miajet_output/HCT116_RAD21-auxin-0hr_intacthic_Guckelberger-2024_ENCFF528XGK_hg38_chr_combined_50Kb/HCT116_RAD21-auxin-0hr_intacthic_Guckelberger-2024_ENCFF528XGK_hg38_chr_combined_saliency-90-p-0.01_50Kb_juicer-visualize.bedpe
	Combining result type: saliency-90
HCT116_RAD21-auxin-0hr_intacthic_Guckelberger-2024_ENCFF528XGK_hg38_chr_combined_50Kb
# HCT116_RAD21-auxin-0hr_intacthic_Guckelberger-2024_